In [36]:
import pandas as pd
import os
import sys

from sapsam import parser, constants  #, detector

import filter_JSON_for_event_labels

csv_paths = parser.get_csv_paths()

# df_raw=pd.read_csv(os.getcwd()+'/models/0.csv', sep=",")
# csv_path = csv_paths[1]
# print(csv_path.name)
df_raw_lst = []
for p in csv_paths:
    df_csv = pd.read_csv(p, header=0, index_col=None)
    df_raw_lst.append(df_csv)
    
df_raw = pd.concat(df_raw_lst, ignore_index=True)

In [37]:
df_raw.head(10)

,Revision ID,Model ID,Organization ID,Datetime,Model JSON,Description,Name,Type,Namespace
0,5ed1535c95a342f4a414dd8c9b1b7df8,1ba74449e9354d0c9d4798b4a6e915d0,6c96d0b6e5e14eeaa9848f953a94e5f8,2019-03-19 11:36:50,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,Pivo,NaN,http://b3mn.org/stencilset/bpmn2.0#
1,301bfb5af53c431a830a6e435de6a590,1ba744cbcafb46c2a52482e36498a9bf,8af3fc1bb4b74e7c8c4089e4921d9056,2020-03-23 17:26:03,"{""resourceId"": ""canvas"", ""formats"": {""document...",NaN,Receipt of Application,NaN,http://b3mn.org/stencilset/bpmn2.0#
2,ac3baa3e59ca476396b4affa8e900c6f,1ba7587b3191422fb03a0e9ba92cebff,36fbc1ee23334e1ab987b07d2108eef2,2017-04-19 06:19:36,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,710 BPMN Conference room,NaN,http://b3mn.org/stencilset/bpmn2.0#
3,da11513a93224fbbb2c34ef9c5cfcaa9,1ba758f87e744ff0bcb3119887cac8cc,574093f96e5e4ff09a8b1f24eeb70284,2021-02-21 13:14:32,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,ECU application p,NaN,http://b3mn.org/stencilset/bpmn2.0#
4,70d06e82d8c344edab8b041f8b03161a,1ba77121c3be44d6bd0b354727f9707b,ff66cd6b779c44bba88ade8952d7e626,2020-06-30 07:12:48,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,2.9.1 Elektronischer Auftrag erfassen,NaN,http://b3mn.org/stencilset/bpmn2.0#
5,8a2b2dc543004b8da056786146882be4,1ba7745757e94a46bc442f143bda9f83,8c91f101cb724ecca985ab2352e62f9b,2017-09-12 10:00:23,"{""resourceId"": ""canvas"", ""formats"": {}, ""ssext...",NaN,Level 2 - Process Area: Human Resources,NaN,http://www.signavio.com/stencilsets/processmap#
6,f027d332958a49ecbe87d2503e1a6cb5,1ba7b02a2f434a4589c1062af97940a3,68e30e8985cc4ccb8b20507ab0ace746,2017-08-20 03:59:54,"{""resourceId"": ""canvas"", ""properties"": {""title...",NaN,New FMC Diagram,NaN,http://www.fmc-modeling.org/stencilsets/fmcblo...
7,e47efaa6dcde4a4586b7ffcb301996d5,1ba7b40cfd804781a4aaa394635d1760,c41660388a04449a995dccf97d894f46,2019-06-12 03:11:21,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,12345678_rev4,NaN,http://b3mn.org/stencilset/bpmn2.0#
8,85a2f746b8844a27ae984694f03e0995,1ba7baf108a4487fa83319991037649c,e2ecf2fb1b7c40a897ede4d3552a4996,2016-02-03 11:45:50,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,Costumes Preparation,NaN,http://b3mn.org/stencilset/bpmn2.0#
9,67ec9dde983b4ee18505e338aee33f0e,1ba7e5ac71e44fe7832cf3202c2544a1,5d8402243c7541e2b6f6f362c8722781,2020-11-10 12:21:46,"{""resourceId"": ""canvas"", ""formats"": {}, ""ssext...",NaN,Level 2 - Process Area: Order Processing,NaN,http://www.signavio.com/stencilsets/processmap#


In [38]:
df = filter_JSON_for_event_labels.add_event_labels_to_DF(df_raw)
# save to pckl file
df.to_pickle("./event_labels.pkl")  
print(df.shape)
df.head(3)

55647it [1:27:30, 10.60it/s] 


KeyboardInterrupt: 

In [ ]:
!pwd

In [ ]:
df.drop(columns=['Revision ID', 'Organization ID', 'Datetime', 'Model JSON', 'Description', 'Type', 'Namespace'], inplace=True)
df.dropna(subset=['event_labels'], inplace=True)

## Language Filter

In [ ]:
## !python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from tqdm import tqdm


def get_df_models_and_labels(df, sep_str=" "):
    df_labels = df.drop(columns="category", axis=1)
    df_labels.reset_index(inplace=True)
    df_labels.drop(columns="element_id", axis=1, inplace=True)
    df_labels.label = df_labels.label.apply(lambda x: str(x or ''))
    df_labels.drop_duplicates(ignore_index=True, inplace=True)
    df_labels['label'] = df_labels.groupby(['model_id'])['label'].transform(lambda x: sep_str.join(x))
    df_labels.drop_duplicates(ignore_index=True, inplace=True)
    return df_labels.set_index("model_id")


def clean(label):
    # handle some special cases
    label = label.replace("\n", " ").replace("\r", "")
    # delete unnecessary whitespaces
    label = label.strip()
    return label


def get_lang_detector(nlp, name):
    return LanguageDetector()


class ModelLanguageDetector:
    def __init__(self, threshold):
        self.threshold = threshold
        self.nlp = spacy.load("en_core_web_sm")
        Language.factory("language_detector", func=get_lang_detector)
        self.nlp.add_pipe('language_detector', last=True)

    def _get_text_language(self, text):
        doc = self.nlp(str(clean(text)))
        detection = doc._.language
        return detection['language']

    def add_detected_natural_language_from_meta(self, df_meta):
        df_meta['detected_natural_language'] = [self._get_text_language(name) for name in tqdm(df_meta.name)]

    def get_detected_natural_language_from_bpmn_model(self, series_txt):
 #       df_labels = get_df_models_and_labels(df, " ")
        # df['detected_natural_language'] = [self._get_text_language(label) for label in tqdm(df['event_labels'])]
        return series_txt.apply(self._get_text_language)


In [ ]:
ld = ModelLanguageDetector(0.8) # comment this line if you have already created the pkl file
df['detected_natural_language'] = ld.get_detected_natural_language_from_bpmn_model(df['event_labels']) # comment this line if you have already created the pkl file
# df.to_pickle(constants.DATA_INTERIM / "all_languages.pkl") # comment this line if you have already created the pkl file
# df = pd.read_pickle(constants.DATA_INTERIM / "bpmn_languages.pkl") # uncomment this line if you have already created the pkl file
df.head()

In [ ]:
df['detected_natural_language'].value_counts()


In [ ]:
df = df[df['detected_natural_language'].isin(['en','ca', 'uk'])]
## todo: check all english language codes

In [ ]:
df.head()